# 01 - Data Understanding and Cleaning

This notebook covers:
- Loading raw data
- Understanding data structure
- Data cleaning and preprocessing
- Saving processed data

🧩 Section 1: Imports & Global Configuration
📌 Purpose

Set up libraries, display options, and plotting defaults so the notebook behaves consistently and remains readable.

In [1]:
# Core data libraries
import pandas as pd
import numpy as np

# Visualization (used lightly in this notebook)
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings control
import warnings
warnings.filterwarnings("ignore")

# Display settings
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: f"{x:.2f}")

# Plot settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)


In [3]:
# File paths (example: using 2 months to start)
file_paths = [
    "C:/Users/vnska/Desktop/NYC_Taxi_Operational_Analytics/data/raw/yellow_tripdata_2022-01.parquet",
    "C:/Users/vnska/Desktop/NYC_Taxi_Operational_Analytics/data/raw/yellow_tripdata_2022-02.parquet"
]

# Load and concatenate
df_list = [pd.read_parquet(fp) for fp in file_paths]
df_raw = pd.concat(df_list, ignore_index=True)

# Basic shape check
df_raw.shape

(5443362, 19)

🧩 Section 3: Initial Data Understanding
📌 Purpose

Understand:

available columns

data types

missing values

basic time coverage

No cleaning yet — only observation.

In [4]:
# Preview data
df_raw.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.00,3.80,1.00,N,142,236,1,14.50,3.00,0.50,3.65,0.00,0.30,21.95,2.50,0.00
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.00,2.10,1.00,N,236,42,1,8.00,0.50,0.50,4.00,0.00,0.30,13.30,0.00,0.00
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.00,0.97,1.00,N,166,166,1,7.50,0.50,0.50,1.76,0.00,0.30,10.56,0.00,0.00
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.00,1.09,1.00,N,114,68,2,8.00,0.50,0.50,0.00,0.00,0.30,11.80,2.50,0.00
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.00,4.30,1.00,N,68,163,1,23.50,0.50,0.50,3.00,0.00,0.30,30.30,2.50,0.00


In [5]:
# Column names and data types
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5443362 entries, 0 to 5443361
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [6]:
# Missing values overview
df_raw.isna().sum().sort_values(ascending=False)

store_and_fwd_flag       173241
RatecodeID               173241
passenger_count          173241
airport_fee              173241
congestion_surcharge     173241
VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
DOLocationID                  0
PULocationID                  0
trip_distance                 0
fare_amount                   0
payment_type                  0
extra                         0
mta_tax                       0
tolls_amount                  0
tip_amount                    0
total_amount                  0
improvement_surcharge         0
dtype: int64

In [7]:
# Time coverage check
df_raw[["tpep_pickup_datetime", "tpep_dropoff_datetime"]].describe()

,tpep_pickup_datetime,tpep_dropoff_datetime
count,5443362,5443362
mean,2022-02-02 01:53:51.905961,2022-02-02 02:08:51.977705
min,2003-01-01 00:10:06,2003-01-01 12:38:59
25%,2022-01-18 23:12:15.250000,2022-01-18 23:26:18.500000
50%,2022-02-03 18:18:27.500000,2022-02-03 18:37:38
75%,2022-02-16 15:01:57,2022-02-16 15:18:05.750000
max,2022-05-24 17:41:50,2022-05-24 17:43:27


🧩 Section 4: Column Selection (Scope Control)
📌 Purpose

Limit the dataset to columns relevant for operational and behavioral analysis.
Dropping noise early improves clarity and performance.

In [8]:
columns_to_keep = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "trip_distance",
    "passenger_count",
    "PULocationID",
    "DOLocationID",
    "payment_type",
    "fare_amount"
]

df = df_raw[columns_to_keep].copy()
df.shape

(5443362, 8)

🧩 Section 5: Datetime Conversion
📌 Purpose

Ensure pickup and dropoff times are in proper datetime format before calculating durations.

In [9]:
df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])

🧩 Section 6: Trip Duration Calculation
📌 Purpose

Create trip duration in minutes — a core metric for all downstream analysis.

In [10]:
df["trip_duration_minutes"] = (
    (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
    .dt.total_seconds() / 60
)

🧩 Section 7: Data Cleaning & Validity Rules
📌 Purpose

Remove records that violate basic physical or logical constraints.

Rules applied:

Trip duration > 0

Trip distance > 0

Passenger count ≥ 0

Reasonable trip duration (< 24 hours)

In [11]:
initial_rows = len(df)

df = df[
    (df["trip_duration_minutes"] > 0) &
    (df["trip_duration_minutes"] < 1440) &
    (df["trip_distance"] > 0) &
    (df["passenger_count"] >= 0)
]

final_rows = len(df)

print(f"Rows before cleaning: {initial_rows}")
print(f"Rows after cleaning:  {final_rows}")

Rows before cleaning: 5443362
Rows after cleaning:  5211380


🧩 Section 8: Speed Sanity Check
📌 Purpose

Detect and remove trips with unrealistic average speeds, which usually indicate GPS or system errors.

In [12]:
# Average speed in km/h (distance is in miles)
df["avg_speed_kmph"] = (df["trip_distance"] * 1.60934) / (df["trip_duration_minutes"] / 60)

# Remove unrealistic speeds
df = df[(df["avg_speed_kmph"] > 0) & (df["avg_speed_kmph"] < 120)]

df["avg_speed_kmph"].describe()

count   5207551.00
mean         19.90
std          10.55
min           0.00
25%          13.39
50%          17.14
75%          22.84
max         119.89
Name: avg_speed_kmph, dtype: float64

🧩 Section 9: Time-Based Feature Engineering
📌 Purpose

Extract reusable temporal features once, so later notebooks stay clean.

In [13]:
df["pickup_hour"] = df["tpep_pickup_datetime"].dt.hour
df["pickup_day"] = df["tpep_pickup_datetime"].dt.day
df["pickup_weekday"] = df["tpep_pickup_datetime"].dt.day_name()
df["pickup_month"] = df["tpep_pickup_datetime"].dt.month

🧩 Section 10: Final Sanity Checks
📌 Purpose

Confirm that cleaned data is statistically reasonable before saving.

In [14]:
df.describe(include="all")

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,passenger_count,PULocationID,DOLocationID,payment_type,fare_amount,trip_duration_minutes,avg_speed_kmph,pickup_hour,pickup_day,pickup_weekday,pickup_month
count,5207551,5207551,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551.00,5207551,5207551.00
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800099,NaN
mean,2022-02-02 01:00:34.324092,2022-02-02 01:15:32.625104,3.11,1.39,165.79,163.70,1.22,12.76,14.97,19.90,14.17,15.51,NaN,1.55
min,2008-12-31 22:23:09,2008-12-31 23:06:56,0.01,0.00,1.00,1.00,1.00,-600.00,0.02,0.00,0.00,1.00,NaN,1.00
25%,2022-01-18 20:32:25,2022-01-18 20:45:08.500000,1.08,1.00,132.00,113.00,1.00,6.50,6.68,13.39,11.00,8.00,NaN,1.00
50%,2022-02-03 17:46:02,2022-02-03 18:04:50,1.77,1.00,162.00,162.00,1.00,9.00,10.70,17.14,15.00,16.00,NaN,2.00
75%,2022-02-16 15:13:35.500000,2022-02-16 15:29:46,3.13,1.00,234.00,234.00,1.00,14.00,16.95,22.84,18.00,23.00,NaN,2.00
max,2022-05-24 17:41:50,2022-05-24 17:43:27,257.70,9.00,265.00,265.00,5.00,401092.32,1439.90,119.89,23.00,31.00,NaN,12.00


🧩 Section 11: Save Cleaned Dataset
📌 Purpose

Persist cleaned, feature-engineered data for reuse in all subsequent notebooks.

In [15]:
output_path = "../data/processed/nyc_taxi_cleaned.parquet"
df.to_parquet(output_path, index=False)

print("Cleaned dataset saved successfully.")

Cleaned dataset saved successfully.
